In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from data import WN18

In [2]:
class Config:
    n_epochs = 100
    batch_size = 150
    embed_dim = 200
    activation = tf.nn.relu
    dropout_rate = 0.2
    maxlen = 5

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""
def glance_dict(d, n=5):
    return dict(itertools.islice(d.items(), n))


def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    triples_all = triples_tr + triples_va + triples_te
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}
    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)
    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    sp2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if (s,p) not in sp2o:
            sp2o[(s,p)] = [o]
        else:
            if o not in sp2o[(s,p)]:
                sp2o[(s,p)].append(o)
    return sp2o


def get_y(triples_tr, e2idx, p2idx, sp2o):
    y = []
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        temp = np.zeros([len(e2idx)])
        temp[sp2o[(s,p)]] = 1.
        y.append(temp)
    y = np.asarray(y)
    return y


def next_train_batch(triples_tr, e2idx, p2idx, sp2o, o2sp):
    context_tr = build_context(triples_tr, o2sp=o2sp)
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i: i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = get_y(_triples_tr, e2idx, p2idx, sp2o)
        _context_tr = context_tr[i: i+Config.batch_size]
        yield (x_s, x_p, y, _context_tr)


def train_input_fn(triples_tr, e2idx, p2idx, sp2o, o2sp):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 sp2o=sp2o,
                                 o2sp=o2sp),
        (tf.int32, tf.int32, tf.float32, tf.int32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)]),
         tf.TensorShape([None, Config.maxlen, 2])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y, context = iterator.get_next()
    return {'s': x_s, 'p': x_p, 'context': context}, y


def build_context_dict(triples_tr):
    temp = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if o not in temp:
            temp[o] = [(s, p)]
        else:
            if (s, p) not in temp[o]:
                temp[o].append((s, p))
    return temp


def build_context(triples, o2sp):
    res = []
    for (_s, _p, _o) in tqdm(triples, total=len(triples), ncols=70):
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o]
        try:
            v = o2sp[s]
        except:
            v = [(len(e2idx), len(p2idx))] * (Config.maxlen)
        if len(v) <= Config.maxlen:
            v += [(len(e2idx), len(p2idx))] * (Config.maxlen - len(v))
        else:
            v = v[:Config.maxlen]
        res.append(v)
    return np.array(res)

In [4]:
def o_next_batch(eval_triples,
                 e2idx, 
                 p2idx,
                 o2sp,):
    context_ev = build_context(eval_triples, o2sp=o2sp)
    for i, (s, p, o) in tqdm(enumerate(eval_triples), total=len(eval_triples), ncols=70):
        s_idx, p_idx = e2idx[s], p2idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx), np.expand_dims(context_ev[i], 0)


def o_input_fn(eval_triples,
               e2idx, 
               p2idx,
               o2sp,):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             e2idx, 
                             p2idx,
                             o2sp),
        (tf.int32, tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,]),
         tf.TensorShape([None, Config.maxlen, 2]),))
    iterator = dataset.make_one_shot_iterator()
    s, p, context = iterator.get_next()
    return {'s': s, 'p': p, 'context': context}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  e2idx,
                  p2idx,
                  o2sp,):
    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               e2idx, 
                               p2idx,
                               o2sp,)))
        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(e2idx)])
        ranks_o, filtered_ranks_o = [], []
        for ((s, p, o), scores_o) in tqdm(zip(eval_triples, ScoresO),
                                          total=len(eval_triples),
                                          ncols=70):
            s_idx, p_idx, o_idx = e2idx[s], p2idx[p], e2idx[o]
            ranks_o += [1 + np.argsort(np.argsort(- scores_o))[o_idx]]
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [e2idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            filtered_ranks_o += [1 + np.argsort(np.argsort(- filtered_scores_o))[o_idx]]
        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(1 / np.asarray(setting_ranks))
            print('[{}] {} MRR: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [5]:
def _forward(x1, x2, masks, is_training):
    x = tf.concat([tf.reduce_mean(x1, axis=1),
                   tf.reduce_sum(x2, axis=1) / tf.reduce_sum(masks, 1, keep_dims=True),],
                  axis=1)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.dense(x, Config.embed_dim, Config.activation)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.layers.dropout(x, 0.2, training=is_training)
    return x

In [ ]:
def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed', [params['e_vocab_size'], Config.embed_dim])
    p_embed = tf.get_variable('p_embed', [params['p_vocab_size'], Config.embed_dim])
    
    e_embed = tf.concat([e_embed, tf.zeros([1, Config.embed_dim])], axis=0)
    p_embed = tf.concat([p_embed, tf.zeros([1, Config.embed_dim])], axis=0)
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    query = tf.stack([s, p], axis=1)
    
    # [batch_sz, maxlen, 2] -> 2 x [batch_sz, maxlen, 1]
    context_s, context_p = tf.split(features['context'], 2, axis=2)
    # [batch_sz, maxlen, 1] -> [batch_sz, maxlen, embed_dim]
    context_s = tf.nn.embedding_lookup(e_embed, tf.squeeze(context_s, -1))
    # [batch_sz, maxlen, 1] -> [batch_sz, maxlen, embed_dim]
    context_p = tf.nn.embedding_lookup(p_embed, tf.squeeze(context_p, -1))
    # [batch_sz, maxlen, embed_dim]
    context = (context_s * context_p)
    # [batch_sz, maxlen+1, embed_dim]
    context = tf.concat([tf.expand_dims(s*p, 1), context], axis=1)
    masks = tf.abs(tf.sign(tf.reduce_sum(context, -1)))
    
    output = _forward(query, context, masks, is_training)
    
    logits = tf.matmul(output, e_embed[:-1, :], transpose_b=True)
    return logits, masks
    
    
def model_fn(features, labels, mode, params):
    logits, masks = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        log_params()
        
        loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                         labels=labels))
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = tf.train.AdamOptimizer().minimize(loss_op,
                                                         global_step=tf.train.get_global_step())
        
        lth = tf.train.LoggingTensorHook({'mask': masks[0]}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op,
                                          training_hooks = [lth])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))


def log_params():
    tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
    tf.logging.info('params: %d'%count_train_params())
    
    
def count_train_params():
    return np.sum([np.prod([d.value for d in v.get_shape()]) for v in tf.trainable_variables()])


def prt_epoch(n_epoch):
    print()
    print("EPOCH %d"%(n_epoch+1))
    print()

In [ ]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
sp2o = build_multi_label(triples_tr)
o2sp = build_context_dict(triples_tr)


model = tf.estimator.Estimator(model_fn,
                               params = params)

for n_epoch in range(Config.n_epochs):
    prt_epoch(n_epoch)
    model.train(lambda: train_input_fn(triples_tr,
                                       e2idx,
                                       p2idx,
                                       sp2o=sp2o,
                                       o2sp=o2sp))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,
                  o2sp=o2sp)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpvg528qr7', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11425b320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

EPOCH 1

INFO:tensorflow:Calling model_fn.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable

100%|█████████████████████| 141442/141442 [00:00<00:00, 397660.86it/s]


INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpvg528qr7/model.ckpt.
INFO:tensorflow:loss = 0.69465816, step = 1
INFO:tensorflow:mask = [1. 1. 1. 1. 0. 0.]
INFO:tensorflow:global_step/sec: 1.98435
INFO:tensorflow:loss = 0.2893965, step = 101 (50.396 sec)
INFO:tensorflow:mask = [1. 1. 1. 1. 0. 0.] (50.395 sec)
INFO:tensorflow:global_step/sec: 2.08213
INFO:tensorflow:loss = 0.07185915, step = 201 (48.028 sec)
INFO:tensorflow:mask = [1. 1. 0. 0. 0. 0.] (48.028 sec)
INFO:tensorflow:global_step/sec: 2.25688
INFO:tensorflow:loss = 0.034814633, step = 301 (44.309 sec)
INFO:tensorflow:mask = [1. 1. 1. 1. 1. 1.] (44.309 sec)
INFO:tensorflow:global_step/sec: 2.13817
INFO:tensorflow:loss = 0.019151809, step = 401 (46.768 sec)
INFO:tensorflow:mask = [1. 1. 1. 0. 0. 0.] (46.768 sec)
INFO:tensorflow:global_step/sec: 1.95154
INFO:tensorflow:loss = 0.013785858, step = 501 (51.242 sec)
INFO:tensorflow:mask = [1. 1. 1. 0. 0. 0.] (51.242 sec)
INFO:tensorf

100%|█████████████████████████████| 5000/5000 [01:48<00:00, 46.01it/s]


[test] Raw MRR: 0.014601583135981434
[test] Raw Hits@1: 0.76
[test] Raw Hits@3: 1.34
[test] Raw Hits@5: 1.7999999999999998
[test] Raw Hits@10: 2.8400000000000003
[test] Filtered MRR: 0.01461612812583305
[test] Filtered Hits@1: 0.76
[test] Filtered Hits@3: 1.34
[test] Filtered Hits@5: 1.7999999999999998
[test] Filtered Hits@10: 2.8400000000000003

EPOCH 2

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(400, 200) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(200,) dtype=float32_ref>]
INFO:tensorflow:params: 

...

INFO:tensorflow:Saving checkpoints for 46208 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpvg528qr7/model.ckpt.
INFO:tensorflow:loss = 8.13982e-06, step = 46208
INFO:tensorflow:mask = [1. 1. 1. 1. 1. 1.]
INFO:tensorflow:global_step/sec: 2.37537
INFO:tensorflow:loss = 4.374127e-06, step = 46308 (42.100 sec)
INFO:tensorflow:mask = [1. 1. 1. 1. 1. 1.] (42.100 sec)
INFO:tensorflow:global_step/sec: 2.38622
INFO:tensorflow:loss = 7.2048215e-06, step = 46408 (41.907 sec)
INFO:tensorflow:mask = [1. 1. 1. 1. 0. 0.] (41.907 sec)
INFO:tensorflow:global_step/sec: 2.39825
INFO:tensorflow:loss = 8.033168e-06, step = 46508 (41.697 sec)
INFO:tensorflow:mask = [1. 1. 1. 1. 0. 0.] (41.697 sec)
INFO:tensorflow:global_step/sec: 2.38753
INFO:tensorflow:loss = 5.6827816e-06, step = 46608 (41.884 sec)
INFO:tensorflow:mask = [1. 1. 1. 1. 1. 1.] (41.884 sec)
INFO:tensorflow:global_step/sec: 2.40663
INFO:tensorflow:loss = 4.2981073e-06, step = 46708 (41.552 sec)
INFO:tensorflow:mask = [1. 1. 1. 0. 0.

100%|█████████████████████████████| 5000/5000 [01:46<00:00, 46.90it/s]


[test] Raw MRR: 0.502503884596597
[test] Raw Hits@1: 38.68
[test] Raw Hits@3: 56.879999999999995
[test] Raw Hits@5: 64.2
[test] Raw Hits@10: 72.48
[test] Filtered MRR: 0.6687483492861743
[test] Filtered Hits@1: 58.64
[test] Filtered Hits@3: 72.44
[test] Filtered Hits@5: 76.92
[test] Filtered Hits@10: 81.39999999999999

EPOCH 51

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(400, 200) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(200,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(200,) dtype=float32_ref>]
INFO:tensorflow:params: 8273600
INFO:tensorflow:Don

100%|█████████████████████| 141442/141442 [00:00<00:00, 292928.32it/s]


INFO:tensorflow:Saving checkpoints for 47151 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpvg528qr7/model.ckpt.
